## 2f. Evidence - Time Behaviour QAS Measurements

Evidence collected in this section checks for the time behaviour QAS scenario defined in the previous step. Note that some functions and data will be loaded from external Python files.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [1]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *
from session_LLMinfo import *

Creating initial custom lists at URI: local:///Users/rbrowersinning/Documents/ResearchFolders/Continuum_LTP/GitRepos/mlte_llm/demo/ReviewPro/../store
Loaded 7 qa_categories for initial list
Loaded 30 quality_attributes for initial list
Creating sample catalog at URI: StoreType.LOCAL_FILESYSTEM:local:///Users/rbrowersinning/Documents/ResearchFolders/Continuum_LTP/GitRepos/mlte_llm/demo/ReviewPro/../store
Loading sample catalog entries.
Loaded 9 entries for sample catalog.


### Set up scenario test case

In [2]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 5
print(card.quality_scenarios[qa].identifier)
print(card.quality_scenarios[qa].quality)
print(
    card.quality_scenarios[qa].stimulus,
    "from ",
    card.quality_scenarios[qa].source,
    " during ",
    card.quality_scenarios[qa].environment,
    ". ",
    card.quality_scenarios[qa].response,
    card.quality_scenarios[qa].measure,
)

card.default-qas_006
Time Behavior
ReviewPro receives a prompt for an employee evaluation from  the manager  during  normal operation .  The time required for the application to retrun an employee evalaution, including a performance score,  should be no more than 10s


### A Specific test case generated from the scenario:

**Data and Data Source:**	The original test data set can be used.

**Measurement and Condition:**	The longest time for a single prompt model response completes is no more than 10 seconds after submission

**Context:**	Normal Operation

### Gather evidence

In [3]:
# import necessary packages
import pandas as pd
import time

from utils.evaluation_helpers import *

In [4]:
# Read the files with with the necessary input data and LLM evaluation results
input_df = pd.read_csv(
    os.path.join(DATASETS_DIR, "2abc_llm_input_functional_correctness.csv")
)


input_df

,employeeSelfEval,managerComments,goalsAndObjectives,EmployeeName,correctEvalScore
0,I need this job to pay my bills and I make cof...,Kate has shown up late to work numerous times ...,Make coffee\nEarn a living,Kate,0
1,"As I reflect on my time working as a barista, ...",Casey has shown up late to work a few times ov...,Make coffee\nEarn a living,Casey,0
2,"Over the past six months, I’ve focused on buil...",Seems more focused on customers than on runnin...,Improve cleanliness of work environment\nLearn...,Charlie,3
3,I have received positive feedback from custome...,Collin has the highest rating in customer sati...,I want to improve my average customer satisfac...,Collin,5
4,"Over the past six months, I’ve focused on buil...",Seems more focused on customers than on runnin...,oals/objectives: - Improve cleanliness of work...,Dave,3
5,"During this review period, I’ve focused on wha...",Goals for Future Development:\nKeep encouragin...,Make coffee\n Earn a living,Beth,4
6,"Over the past review period, I’ve really enjoy...",Strengths:\nAnnabeth always shows up to the co...,Make coffee\n Earn a living,Annabeth,4


### Save evidence to the specicified scenario

In [5]:
chain = prompt_template | llm
time_performances = []
chat_responses = []


def time_model(input_df):
    for index, row in input_df.iterrows():
        pii_data = {
            "employee_name": row.EmployeeName,
            "goals_and_objectives": row.goalsAndObjectives,
            "self_eval": row.employeeSelfEval,
            "manager_comments": row.managerComments,
        }

    prompt = prompt_template.format(**pii_data)

    start_time = time.time()
    chat_response = chain.invoke(pii_data)
    time_performances.append(time.time() - start_time)

    return time_performances

In [6]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the TestSuite.
time_measurement = ExternalMeasurement(
    "results returned promptly", Array, time_model
)
time_res = time_measurement.evaluate(input_df)

# Inspect value
print(time_res)

# Save to artifact store
time_res.save(force=True)

[16.129170894622803]


ArtifactModel(header=ArtifactHeaderModel(identifier='evidence.results returned promptly', type='evidence', timestamp=1761930190, creator=None, level='version'), body=EvidenceModel(artifact_type=<ArtifactType.EVIDENCE: 'evidence'>, metadata=EvidenceMetadata(test_case_id='results returned promptly', measurement=MeasurementMetadata(measurement_class='mlte.measurement.external_measurement.ExternalMeasurement', output_class='mlte.evidence.types.array.Array', additional_data={'function': '__main__.time_model'})), evidence_class='mlte.evidence.types.array.Array', value=ArrayValueModel(evidence_type=<EvidenceType.ARRAY: 'array'>, data=[16.129170894622803])))